## Evaluation of the Solution Space under different conditions

In [ ]:
# Ensure necessary packages are installed
!pip install cobra efmtool numpy pandas scipy

COBRApy Definition of the SK Network from Szallasi's "A first course in Systems Biology"

In [1]:
from cobra import Model, Reaction, Metabolite

# Modell erstellen
model = Model("Simple_Network")

# Metaboliten definieren
A_EX = Metabolite("A_EX", compartment="e")
B_EX = Metabolite("B_EX", compartment="e")
P_EX = Metabolite("P_EX", compartment="e")
E_EX = Metabolite("E_EX", compartment="e")
A = Metabolite("A", compartment="c")
B = Metabolite("B", compartment="c")
C = Metabolite("C", compartment="c")
D = Metabolite("D", compartment="c")
E = Metabolite("E", compartment="c")
P = Metabolite("P", compartment="c")


# Reaktionen definieren
r1 = Reaction("A_in")
r1.name = "r1"
r1.lower_bound = 0.0
r1.upper_bound = 1000.0
r1.add_metabolites({A_EX: -1, A: 1})

r2 = Reaction("B_in_out")
r2.name = "r2"
r2.lower_bound = -1000.0
r2.upper_bound = 1000.0
r2.add_metabolites({B_EX: -1, B: 1})

r3 = Reaction("P_out")
r3.name = "r3"
r3.lower_bound = 0.0
r3.upper_bound = 1000.0
r3.add_metabolites({P: -1, P_EX: 1})

r4 = Reaction("E_out")
r4.name = "r4"
r4.lower_bound = 0.0
r4.upper_bound = 1000.0
r4.add_metabolites({E_EX: 1, E: -1})

r5 = Reaction("r5")
r5.name = "r5"
r5.lower_bound = 0.0
r5.upper_bound = 1000.0
r5.add_metabolites({A: -1, B: 1})

r6 = Reaction("r6")
r6.name = "r6"
r6.lower_bound = 0.0
r6.upper_bound = 1000.0
r6.add_metabolites({A: -1, C: 1})

r7 = Reaction("r7")
r7.name = "r7"
r7.lower_bound = 0.0
r7.upper_bound = 1000.0
r7.add_metabolites({A: -1, D: 1})

r8 = Reaction("r8")
r8.name = "r8"
r8.lower_bound = -1000.0
r8.upper_bound = 1000.0
r8.add_metabolites({B: -1, C: 1})

r9 = Reaction("r9")
r9.name = "r9"
r9.lower_bound = 0.0
r9.upper_bound = 1000.0
r9.add_metabolites({B: -1, P: 1})

r10 = Reaction("r10")
r10.name = "r10"
r10.lower_bound = 0.0
r10.upper_bound = 1000.0
r10.add_metabolites({C: -1, D: -1, P: 1, E: 1})


# Reaktionen zum Modell hinzufügen
model.add_reactions([r1, r2, r3, r4, r5, r6, r7, r8, r9, r10])

# Modell überprüfen
print("Reaktionen:", [rxn.id for rxn in model.reactions])
print("Metaboliten:", [met.id for met in model.metabolites])
print("Gene:", [gene.id for gene in model.genes])

# Modell speichern (optional)
#model.save("simple_network.xml")


Reaktionen: ['A_in', 'B_in_out', 'P_out', 'E_out', 'r5', 'r6', 'r7', 'r8', 'r9', 'r10']
Metaboliten: ['A_EX', 'A', 'B_EX', 'B', 'P', 'P_EX', 'E_EX', 'E', 'C', 'D']
Gene: []


In [2]:
import numpy as np
import cobra


def get_stoichiometric_matrix(model):
    # Filtere die Reaktionen, um nur Reaktionen zu behalten, die nicht mit "EX_" beginnen
    internal_reactions = [rxn for rxn in model.reactions if not rxn.id.startswith("EX_")]

    # Extrahiere die Metaboliten aus dem Modell
    internal_metabolites = [meta for meta in model.metabolites if not meta.compartment == 'e']

    # Erstelle die leere Stöchiometrie-Matrix mit (Anzahl der Metaboliten, Anzahl der internen Reaktionen)
    stoichiometric_matrix = np.zeros((len(internal_metabolites), len(internal_reactions)))

    # Befülle die Stöchiometrie-Matrix so, dass jede Spalte eine Reaktion repräsentiert
    for j, rxn in enumerate(internal_reactions):  # Spaltenweise über Reaktionen iterieren
        for i, met in enumerate(internal_metabolites):  # Zeilenweise über Metaboliten iterieren
            stoichiometric_matrix[i, j] = rxn.metabolites.get(met, 0)  # Falls Metabolit nicht in der Reaktion ist, wird 0 zurückgegeben
    return stoichiometric_matrix


In [3]:
import efmtool
import cobra
import numpy as np
from efmtool import calculate_efms

def cobra_to_efms(model):
    # 1. Stoichiometric matrix (S matrix)
    metabolites = [
    metabolite for metabolite in model.metabolites 
    if metabolite.compartment == "c"
    ]
    reactions = model.reactions
    stoichiometry = np.zeros((len(metabolites), len(reactions)))
    #stoichiometry = get_stoichiometric_matrix(model)
    
    for i, met in enumerate(metabolites):
        for j, rxn in enumerate(reactions):
            try:
                stoichiometry[i, j] = rxn.get_coefficient(met.id)
            except KeyError:
                
                stoichiometry[i, j] = 0.0
    
    print("Stoichiometric matrix shape:", stoichiometry.shape)
    print("Stoichiometric matrix:", stoichiometry)

    # 2. Reversibilities: 1 for reversible, 0 for irreversible reactions
    reversibilities = [1 if rxn.reversibility else 0 for rxn in reactions]
    
    print("Reversibilities:", reversibilities)
    print("Number of reactions:", len(reactions))

    # 3. Reaction names
    reaction_names = [rxn.id for rxn in reactions]
    
    # 4. Metabolite names
    metabolite_names = [met.id for met in metabolites]
    
    return calculate_efms(stoichiometry, reversibilities, reaction_names, metabolite_names), reaction_names





# 6. Berechne die Elementarmoden (EFMs) mit dem efmtool
efms, reaction_names = cobra_to_efms(model)

# 7. Ergebnisse anzeigen
print("Berechnete Elementarmoden:")
print(np.column_stack((reaction_names, efms)))


Stoichiometric matrix shape: (6, 10)
Stoichiometric matrix: [[ 1.  0.  0.  0. -1. -1. -1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.  0.  0. -1. -1.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.  0.  1.  1.]
 [ 0.  0.  0. -1.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.  0.  1.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0. -1.]]
Reversibilities: [0, 1, 0, 0, 0, 0, 0, 1, 0, 0]
Number of reactions: 10
2025-01-02  22:48:01.518  main                     INFO     | logger initialized
2025-01-02  22:48:01.519  main                     INFO     | =====================================================
2025-01-02  22:48:01.519  main                     INFO     | efmtool version 4.7.1, 2009-12-04 18:30:05
2025-01-02  22:48:01.519  main                     INFO     | Copyright (c) 2009, Marco Terzer, Zurich, Switzerland
2025-01-02  22:48:01.519  main                     INFO     | This is free software, !!! NO WARRANTY !!!
2025-01-02  22:48:01.519  main                     INFO     | See LICENCE.txt for redi

In [4]:
import numpy as np


input_array = np.column_stack((reaction_names, efms))

# Extract headers and values
headers = input_array[:, 0]
values = input_array[:, 1:].astype(float)  # Convert values to float

# Construct dictionaries
experimental_data = []
for col_idx in range(values.shape[1]):
    data_point = {}
    for row_idx, header in enumerate(headers):
        if header.endswith("_in") or header.endswith("_out"):
            # Add only reactions ending with "_in" or "_out"
            data_point[header] = values[row_idx, col_idx]
    # Normalize so "A_in" is always 1.0
    if "A_in" in data_point and data_point["A_in"] != 0.0:
        normalization_factor = data_point["A_in"]
        for key in data_point.keys():
            data_point[key] /= normalization_factor
    experimental_data.append(data_point)

# Output the result
for data in experimental_data:
    print(data)

experimental_data_zeropadded_out = experimental_data

{'A_in': 1.0, 'B_in_out': 0.0, 'P_out': 0.5, 'E_out': 0.5}
{'A_in': 1.0, 'B_in_out': -1.0, 'P_out': 0.0, 'E_out': 0.0}
{'A_in': 1.0, 'B_in_out': 0.0, 'P_out': 1.0, 'E_out': 0.0}
{'A_in': 1.0, 'B_in_out': 0.0, 'P_out': 1.0, 'E_out': 0.0}
{'A_in': 1.0, 'B_in_out': -1.0, 'P_out': 0.0, 'E_out': 0.0}
{'A_in': 1.0, 'B_in_out': 0.0, 'P_out': 0.5, 'E_out': 0.5}
{'A_in': 1.0, 'B_in_out': 1.0, 'P_out': 1.0, 'E_out': 1.0}
{'A_in': 0.0, 'B_in_out': 1.0, 'P_out': 1.0, 'E_out': 0.0}


In [5]:
import numpy as np
from scipy.optimize import lsq_linear
from scipy.optimize import minimize





#print(reaction_ids)
# Funktion zur Ableitung des Zielvektors aus den experimentellen Daten
def create_target_vector(data_point, reaction_ids):
    target_flux = np.full(len(reaction_ids), np.nan)  # Vektor mit NaN initialisieren
    for rxn_id, value in data_point.items():
        if rxn_id in reaction_ids:  # Nur Reaktionen, die im Modell existieren
            idx = reaction_ids.index(rxn_id)
            target_flux[idx] = value  # Setze den Zielwert
    return target_flux


def least_square_min(model, S, experimental_data):
    reaction_ids = [rxn.id for rxn in model.reactions if not rxn.id.startswith("EX_")]
    total_score = 0.0
    successful_optimizations = 0

    for data_point in experimental_data:
        # Zielvektor für aktuellen Datenpunkt erstellen
        target_flux = create_target_vector(data_point, reaction_ids)

        # Filter für bekannte Flüsse (Reaktionen, die in 'target_flux' nicht NaN sind)
        known_flux_indices = ~np.isnan(target_flux)
        target_flux_values = target_flux[known_flux_indices]

        def objective(x):
            return np.linalg.norm(x[known_flux_indices] - target_flux_values)

        constraints = {'type': 'eq', 'fun': lambda x: np.dot(S, x)}

        initial_guess = np.zeros(len(reaction_ids))
        bounds = [(model.reactions.get_by_id(rxn_id).lower_bound,
                   model.reactions.get_by_id(rxn_id).upper_bound)
                  for rxn_id in reaction_ids]

        result = minimize(objective, initial_guess, constraints=constraints, bounds=bounds, tol=1e-10)

        if result.success:
            optimized_flux = result.x
            total_score += np.linalg.norm(optimized_flux[known_flux_indices] - target_flux_values)
            successful_optimizations += 1
        else:
            print(f"Optimization failed for data point: {data_point}. Message: {result.message}")

    # Return None if any optimization failed, otherwise the total score
    if successful_optimizations < len(experimental_data):
        return 0.0, len(experimental_data)
    else:
        return total_score, len(experimental_data)

 
S = get_stoichiometric_matrix(model)
print(least_square_min(model, S, experimental_data_zeropadded_out))

(9.487838378972288e-07, 8)


In [55]:
import random
from cobra import Reaction

# Erstellen von Reaktionen mit 1-2 Substrat(en) und 1-2 Produkt(en)
def generate_random_reactions(model, num_reactions=2000):
    # Filter out metabolites that end with '_EX'
    metabolites = [met for met in model.metabolites if not met.compartment == 'e']
    random_reactions = []

    for i in range(num_reactions):
        reaction_valid = False

        while not reaction_valid:
            # Zufällige Auswahl der Anzahl an Substraten und Produkten
            num_reactants = random.randint(1, 2)
            num_products = random.randint(1, 2)

            # Zufällige Auswahl der Substrate und Produkte
            reactants = random.sample(metabolites, num_reactants)
            products = random.sample(metabolites, num_products)

            # Check, dass auf beiden Seiten Metaboliten vorhanden sind
            if set(reactants).isdisjoint(set(products)):
                reaction_valid = True

                # Erstellung der Reaktion
                reaction = Reaction(f'Random_Rxn_{i+1}')
                reaction.name = f'Random Reaction {i+1}'

                reaction.add_metabolites({
                    met: -1.0 for met in reactants
                })
                reaction.add_metabolites({
                    met: 1.0 for met in products
                })

                # 50% Chance, die Reaktion reversibel zu machen
                if random.random() < 0.5:
                    reaction.lower_bound = -1000.0
                else:
                    reaction.lower_bound = 0.0

                reaction.upper_bound = 1000.0

                random_reactions.append(reaction)

    return random_reactions

# Generierung von 2000 zufälligen Reaktionen + den Inversen
random_reactions_list = generate_random_reactions(model, num_reactions=2000)

# Anzahl der insgesamt generierten Reaktionen
print(f"\nTotal number of random reactions generated: {len(random_reactions_list)}")


Total number of random reactions generated: 2000


In [7]:
import cobra
import random

def sample_reactions(model, n_reactions):
  
    # Filter reactions that do not end with '_in' or '_out'
    valid_reactions = [rxn for rxn in model.reactions if not rxn.id.endswith('_in') and not rxn.id.endswith('_out')]

    # Ensure that the model has enough valid reactions
    if n_reactions > len(valid_reactions):
        raise ValueError("Requested number of reactions exceeds the total number of valid reactions in the model")

    # Sample reactions randomly from the valid reactions list
    sampled_reactions = random.sample(valid_reactions, n_reactions)

    for rxn in sampled_reactions:
        print(rxn.id)
    
    return sampled_reactions


In [8]:
import random

def sample_reactions_from_list(reactions_list, n_reactions):
    
    # Ensure the list has enough reactions to sample from
    if n_reactions > len(reactions_list):
        raise ValueError("Requested number of reactions exceeds the total number of reactions in the list")

    # Sample reactions randomly from the given list
    sampled_reactions = random.sample(reactions_list, n_reactions)

    for rxn in sampled_reactions:
        print(rxn.id if hasattr(rxn, 'id') else rxn)  # Print the ID if the object has one

    return sampled_reactions


In [9]:
def model_edit_distance(model1, model2):
   
    # Get sets of reaction IDs for both models
    reactions_model1 = {rxn.id for rxn in model1.reactions}
    reactions_model2 = {rxn.id for rxn in model2.reactions}
    
    # Calculate the symmetric difference (reactions present in one model but not the other)
    differing_reactions = reactions_model1.symmetric_difference(reactions_model2)
    
    # The edit distance is the number of differing reactions
    edit_distance = len(differing_reactions)
    
    return edit_distance




In [10]:
import pandas as pd

def filter_and_display(list1, list2, list3):
  
    # Erstellen des DataFrames
    df = pd.DataFrame({'Scores': list1, 'Edit Distances': list2, 'Computation Count': list3})
    
    # Filtern: Entfernen von Zeilen mit NaN oder 0.0 in der ersten Liste
    filtered_df = df.dropna(subset=['Scores'])
    filtered_df = filtered_df[filtered_df['Scores'] != 0.0]
    
    # Ergebnis zurückgeben oder anzeigen
    return filtered_df




## Generation of data points using sampling

In [11]:
import cobra
from cobra.sampling import sample

import pandas as pd


# Function to normalize a row based on A_in, excluding boundary reactions
def normalize_row(row, normalization_id):
    normalization_factor = row[normalization_id]
    # Exclude boundary reactions (columns starting with "EX_" or ending with "_out")
    filtered_row = {col: value for col, value in row.items() if (col.endswith("_in") or col.endswith("_out"))}
    return {col: (value / normalization_factor) if normalization_factor != 0 else 0.0
            for col, value in filtered_row.items()}


def create_synthetic_datapoints(model, amount, normalization_id):
    s = sample(model, amount)
    
    normalized_data = [normalize_row(row, normalization_id) for _, row in s.iterrows()]
    
    return normalized_data

In [12]:
model_data_gen = model.copy()

# Exchange Reaktionen für extracelluläre Metaboliten
model_data_gen.add_boundary(A_EX, type="exchange")
model_data_gen.add_boundary(E_EX, type="exchange")
model_data_gen.add_boundary(B_EX, type="exchange")
model_data_gen.add_boundary(P_EX, type="exchange")

Reaction identifier,EX_P_EX
Name,exchange
Memory address,0x28ba4d410
Stoichiometry,P_EX <=> <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [13]:
import cobra

def check_for_deadends(model):    

    # Lists for results
    upstream_no_consumption_roots = []
    downstream_no_production_roots = []

    # Initialize dictionaries for production and consumption of metabolites
    metabolite_produced = {met: False for met in model.metabolites if met.compartment == 'c'}
    metabolite_consumed = {met: False for met in model.metabolites if met.compartment == 'c'}

    # Iterate over all reactions in the model
    for reaction in model.reactions:
        if reaction.reversibility:
            #print(reaction)
            # If reversible, treat all metabolites as both reactants and products
            for met in reaction.metabolites:
                metabolite_produced[met] = True
                metabolite_consumed[met] = True
        else:
            # Metabolites produced in this reaction
            for product in reaction.products:
                #print(reaction.products)
                if product.compartment == 'c':  # Only consider intracellular metabolites
                    metabolite_produced[product] = True

            # Metabolites consumed in this reaction
            for reactant in reaction.reactants:
                
                if reactant.compartment == 'c':  # Only consider intracellular metabolites
                    metabolite_consumed[reactant] = True

    # Determine Upstream No-Consumption Roots (produced but not consumed)
    for met, produced in metabolite_produced.items():
        if produced and not metabolite_consumed[met]:
            upstream_no_consumption_roots.append(met.id)

    # Determine Downstream No-Production Roots (consumed but not produced)
    for met, consumed in metabolite_consumed.items():
        if consumed and not metabolite_produced[met]:
            downstream_no_production_roots.append(met.id)

    #print("Upstream No-Consumption Roots:", upstream_no_consumption_roots)
    #print("Downstream No-Production Roots:", downstream_no_production_roots)

    return len(upstream_no_consumption_roots) + len(downstream_no_production_roots)




In [14]:
def filter_datapoints(datapoints, reaction_list):
 
    
    ids = [reaction.id for reaction in reaction_list]
    
    filtered_data = []

    for datapoint in datapoints:
        filtered_datapoint = {key: value for key, value in datapoint.items() if key in ids}
        filtered_data.append(filtered_datapoint)

    return filtered_data

In [15]:
from itertools import combinations

def evaluate_reaction_subsets(model, experimental_data, random_reactions, num_removed):
  
    
    threshold = len(experimental_data)/10 * 3e-7
    
    # Step 1: Remove 2 reactions from the model
    model_copy = model.copy()
    valid_reactions = [rxn for rxn in model_copy.reactions if not rxn.id.endswith('_in') and not rxn.id.endswith('_out')]
    removed_reactions = random.sample(valid_reactions, num_removed)
    model_copy.remove_reactions(removed_reactions)
    print(model_copy.reactions)

    experimental_data = filter_datapoints(experimental_data, model_copy.reactions)
    
    # Step 2: Assemble reaction pool of reactions that can be added
    reaction_pool = removed_reactions + random_reactions

    # Step 3: Evaluate subsets of size 2 and 3
    adequate_subsets_count = 0
    num_subsets_unblocked = 0
    for subset_size in [2, 3]:
        for subset in combinations(reaction_pool, subset_size):
            # Add the subset of reactions to the model
            temp_model = model_copy.copy()
            temp_model.add_reactions(subset)

            # Evaluate the score
            S = get_stoichiometric_matrix(temp_model)
            score, _ = least_square_min(temp_model, S, experimental_data)
            
            # Check if the score meets the criteria
            if score < threshold and score != 0.0:
                adequate_subsets_count += 1
            
                num_deadends = check_for_deadends(temp_model)
                if num_deadends == 0:
                    num_subsets_unblocked += 1
                
    return adequate_subsets_count, num_subsets_unblocked



In [56]:
random.seed(42)
np.random.seed(42)

In [57]:
test_random_reacs = generate_random_reactions(model, 13)
#remove_reacs = sample_reactions(model, 2)

In [58]:
random.seed(42)
np.random.seed(42)

In [59]:
synth_data20 = create_synthetic_datapoints(model_data_gen, 20, "A_in")

In [60]:
random.seed(42)
np.random.seed(42)

In [61]:
synth_data100 = create_synthetic_datapoints(model_data_gen, 100, "A_in")

In [62]:
random.seed(42)
np.random.seed(42)

In [63]:
synth_data500 = create_synthetic_datapoints(model_data_gen, 500, "A_in")

In [64]:
random.seed(17)
np.random.seed(17)

## Evaluating number of adequate subsets for different experimental data

In [65]:
num_adequate_subsets_efm, num_unblocked_efm = evaluate_reaction_subsets(model, experimental_data_zeropadded_out, test_random_reacs, 2)

[<Reaction A_in at 0x28c7a1450>, <Reaction B_in_out at 0x28c740f90>, <Reaction P_out at 0x28c7a1490>, <Reaction E_out at 0x28c7a1510>, <Reaction r5 at 0x28c7a1550>, <Reaction r6 at 0x28c7a14d0>, <Reaction r7 at 0x28c7a1650>, <Reaction r10 at 0x28c7a1590>]


In [66]:
print(f"Number of subsets with adequate scores: {num_adequate_subsets_efm}")
print(f"Number of unblocked models: {num_unblocked_efm}")

Number of subsets with adequate scores: 143
Number of unblocked models: 143


In [67]:
random.seed(17)
np.random.seed(17)

In [68]:
num_adequate_subsets20, num_unblocked_20 = evaluate_reaction_subsets(model, synth_data20, test_random_reacs, 2)
print(f"Number of subsets with adequate scores: {num_adequate_subsets20}")
print(f"Number of unblocked models: {num_unblocked_20}")

[<Reaction A_in at 0x1730ec590>, <Reaction B_in_out at 0x1730d5010>, <Reaction P_out at 0x1730d6e10>, <Reaction E_out at 0x1730d4590>, <Reaction r5 at 0x1731ba650>, <Reaction r6 at 0x1731eda90>, <Reaction r7 at 0x17310efd0>, <Reaction r10 at 0x28d8b8910>]
Number of subsets with adequate scores: 143
Number of unblocked models: 143


In [69]:
random.seed(17)
np.random.seed(17)

In [70]:
num_adequate_subsets100, num_unblocked_100 = evaluate_reaction_subsets(model, synth_data100, test_random_reacs, 2)
print(f"Number of subsets with adequate scores: {num_adequate_subsets100}")
print(f"Number of unblocked models: {num_unblocked_100}")

[<Reaction A_in at 0x28f30c890>, <Reaction B_in_out at 0x28d82b710>, <Reaction P_out at 0x28c14e090>, <Reaction E_out at 0x28c1364d0>, <Reaction r5 at 0x28c1361d0>, <Reaction r6 at 0x28c22b910>, <Reaction r7 at 0x28c22b950>, <Reaction r10 at 0x28c229e50>]
Number of subsets with adequate scores: 145
Number of unblocked models: 145


In [71]:
random.seed(17)
np.random.seed(17)

In [72]:
num_adequate_subsets500, num_unblocked_500 = evaluate_reaction_subsets(model, synth_data500, test_random_reacs, 2)
print(f"Number of subsets with adequate scores: {num_adequate_subsets500}")
print(f"Number of unblocked models: {num_unblocked_500}")

[<Reaction A_in at 0x28d82aad0>, <Reaction B_in_out at 0x28d82aa50>, <Reaction P_out at 0x28d82b1d0>, <Reaction E_out at 0x28d82a650>, <Reaction r5 at 0x28c98a450>, <Reaction r6 at 0x28c988dd0>, <Reaction r7 at 0x28c98a050>, <Reaction r10 at 0x28c98bf90>]
Number of subsets with adequate scores: 146
Number of unblocked models: 146


Different Seed

In [73]:
random.seed(23)
np.random.seed(23)

In [74]:
num_adequate_subsets_efm23, num_unblocked_efm_23 = evaluate_reaction_subsets(model, experimental_data_zeropadded_out, test_random_reacs, 2)
print(f"Number of subsets with adequate scores: {num_adequate_subsets_efm23}")
print(f"Number of unblocked models: {num_unblocked_efm_23}")

[<Reaction A_in at 0x28cdf8590>, <Reaction B_in_out at 0x28cdf97d0>, <Reaction P_out at 0x28cdf95d0>, <Reaction E_out at 0x28cdfa0d0>, <Reaction r6 at 0x28cbd2750>, <Reaction r8 at 0x28cbd1e50>, <Reaction r9 at 0x28cbd3590>, <Reaction r10 at 0x28cbd1290>]
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.0, 'P_out': 0.5, 'E_out': 0.5}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -1.0, 'P_out': 0.0, 'E_out': 0.0}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.0, 'P_out': 1.0, 'E_out': 0.0}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.0, 'P_out': 1.0, 'E_out': 0.0}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -1.0, 'P_out': 0.0, 'E_out': 0.0}. Message: Inequality constraints incompatible
Optimization failed for data point: {

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.0, 'P_out': 0.5, 'E_out': 0.5}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.0, 'P_out': 1.0, 'E_out': 0.0}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.0, 'P_out': 1.0, 'E_out': 0.0}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.0, 'P_out': 0.5, 'E_out': 0.5}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 1.0, 'P_out': 1.0, 'E_out': 1.0}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 0.0, 'B_in_out': 1.0, 'P_out': 1.0, 'E_out': 0.0}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.0, 'P_out': 0.5, 'E_out': 0.5}. Message: Inequality constraints incompatible
Optimization 

In [75]:
random.seed(23)
np.random.seed(23)

In [76]:
num_adequate_subsets20_23, num_unblocked_20_23 = evaluate_reaction_subsets(model, synth_data20, test_random_reacs, 2)
print(f"Number of subsets with adequate scores: {num_adequate_subsets20_23}")
print(f"Number of unblocked models: {num_unblocked_20_23}")

[<Reaction A_in at 0x28f49f4d0>, <Reaction B_in_out at 0x28f49efd0>, <Reaction P_out at 0x28f49d790>, <Reaction E_out at 0x28f49e7d0>, <Reaction r6 at 0x28f49e510>, <Reaction r8 at 0x28f49ff50>, <Reaction r9 at 0x28f49e210>, <Reaction r10 at 0x28f49e110>]
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.17556652028137912, 'P_out': 0.7959359083105236, 'E_out': 0.37963061197085474}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.12695949927258776, 'P_out': 0.7996891902527202, 'E_out': 0.0733513104746905}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.1687351147790975, 'P_out': 1.0135504931721213, 'E_out': 0.15518462160697422}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.03000809010872362, 'P_out': 0.9986189628140699, 'E_out': 0.03138912729465204}. Message: Inequality constraints incomp

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.17556652028137912, 'P_out': 0.7959359083105236, 'E_out': 0.37963061197085474}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.1687351147790975, 'P_out': 1.0135504931721213, 'E_out': 0.15518462160697422}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.03000809010872362, 'P_out': 0.9986189628140699, 'E_out': 0.03138912729465204}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 1.146112335377967, 'P_out': 2.117941604252662, 'E_out': 0.028170731125300077}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.4576318766822288, 'P_out': 0.9799357291644663, 'E_out': 0.4776961475177655}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.0

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.17556652028137912, 'P_out': 0.7959359083105236, 'E_out': 0.37963061197085474}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.12695949927258776, 'P_out': 0.7996891902527202, 'E_out': 0.0733513104746905}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.1687351147790975, 'P_out': 1.0135504931721213, 'E_out': 0.15518462160697422}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.03000809010872362, 'P_out': 0.9986189628140699, 'E_out': 0.03138912729465204}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 1.146112335377967, 'P_out': 2.117941604252662, 'E_out': 0.028170731125300077}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.17556652028137912, 'P_out': 0.7959359083105236, 'E_out': 0.37963061197085474}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.12695949927258776, 'P_out': 0.7996891902527202, 'E_out': 0.0733513104746905}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.1687351147790975, 'P_out': 1.0135504931721213, 'E_out': 0.15518462160697422}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.03000809010872362, 'P_out': 0.9986189628140699, 'E_out': 0.03138912729465204}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 1.146112335377967, 'P_out': 2.117941604252662, 'E_out': 0.028170731125300077}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 1.146112335377967, 'P_out': 2.117941604252662, 'E_out': 0.028170731125300077}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.5310271914517872, 'P_out': 0.4032138862726843, 'E_out': 0.06575892227552736}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.17369150679527035, 'P_out': 0.432054634267663, 'E_out': 0.39425385893706144}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.17556652028137912, 'P_out': 0.7959359083105236, 'E_out': 0.37963061197085474}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.12695949927258776, 'P_out': 0.7996891902527202, 'E_out': 0.0733513104746905}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 

In [77]:
random.seed(23)
np.random.seed(23)

In [78]:
num_adequate_subsets100_23, num_unblocked_100_23 = evaluate_reaction_subsets(model, synth_data100, test_random_reacs, 2)
print(f"Number of subsets with adequate scores: {num_adequate_subsets100_23}")
print(f"Number of unblocked models: {num_unblocked_100_23}")

[<Reaction A_in at 0x28c770d50>, <Reaction B_in_out at 0x28d883390>, <Reaction P_out at 0x172ed8f90>, <Reaction E_out at 0x28c772850>, <Reaction r6 at 0x28c773ad0>, <Reaction r8 at 0x28d91cc10>, <Reaction r9 at 0x28d91fe90>, <Reaction r10 at 0x28c773c50>]
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.24086596707530541, 'P_out': 0.918007811738909, 'E_out': 0.3228581553363943}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.1495988856302185, 'P_out': 0.620000349869832, 'E_out': 0.230400764499948}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.5440575791038958, 'P_out': 0.9660930999597694, 'E_out': 0.577964479144121}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.27759661828070736, 'P_out': 1.2202088609154078, 'E_out': 0.05738775736529739}. Message: Inequality constraints incompatible


Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.24086596707530541, 'P_out': 0.918007811738909, 'E_out': 0.3228581553363943}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.5440575791038958, 'P_out': 0.9660930999597694, 'E_out': 0.577964479144121}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 1.5757525991017556, 'P_out': 2.0726183308437425, 'E_out': 0.5031342682580073}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.1428668227373649, 'P_out': 0.5716089506263634, 'E_out': 0.2855242266362729}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.27759661828070736, 'P_out': 1.2202088609154078, 'E_out': 0.05738775736529739}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.6755

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.24086596707530541, 'P_out': 0.918007811738909, 'E_out': 0.3228581553363943}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.1495988856302185, 'P_out': 0.620000349869832, 'E_out': 0.230400764499948}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.5440575791038958, 'P_out': 0.9660930999597694, 'E_out': 0.577964479144121}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 1.5757525991017556, 'P_out': 2.0726183308437425, 'E_out': 0.5031342682580073}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.1428668227373649, 'P_out': 0.5716089506263634, 'E_out': 0.2855242266362729}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.2775966

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.3289450668707375, 'P_out': 0.7860393327377471, 'E_out': 0.5429057341329694}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.28331544771859896, 'P_out': 0.6436300030489995, 'E_out': 0.07305454923228279}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.8162184322962625, 'P_out': 0.11139961582049077, 'E_out': 0.07238195188328878}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.2778598543018742, 'P_out': 1.1116425239049164, 'E_out': 0.1662173303969951}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.5930315530470205, 'P_out': 0.8739596377832364, 'E_out': 0.7190719152637556}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.6755428454384402, 'P_out': 1.4399010728075883, 'E_out': 0.23564177263084984}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.22633333634941785, 'P_out': 0.7405282491628188, 'E_out': 0.03313841448772367}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.57557376580066, 'P_out': 0.4057801237358398, 'E_out': 0.018646110463459133}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.24086596707530541, 'P_out': 0.918007811738909, 'E_out': 0.3228581553363943}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.1495988856302185, 'P_out': 0.620000349869832, 'E_out': 0.230400764499948}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.54

...

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.24086596707530541, 'P_out': 0.918007811738909, 'E_out': 0.3228581553363943}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.5440575791038958, 'P_out': 0.9660930999597694, 'E_out': 0.577964479144121}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 1.5757525991017556, 'P_out': 2.0726183308437425, 'E_out': 0.5031342682580073}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.1428668227373649, 'P_out': 0.5716089506263634, 'E_out': 0.2855242266362729}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.27759661828070736, 'P_out': 1.2202088609154078, 'E_out': 0.05738775736529739}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 1.0759

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.24086596707530541, 'P_out': 0.918007811738909, 'E_out': 0.3228581553363943}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.1495988856302185, 'P_out': 0.620000349869832, 'E_out': 0.230400764499948}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.5440575791038958, 'P_out': 0.9660930999597694, 'E_out': 0.577964479144121}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 1.5757525991017556, 'P_out': 2.0726183308437425, 'E_out': 0.5031342682580073}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.1428668227373649, 'P_out': 0.5716089506263634, 'E_out': 0.2855242266362729}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.2775966

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.4468454540870484, 'P_out': 0.4179355745721925, 'E_out': 0.13521897134078664}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.5777421951491593, 'P_out': 0.2602968371819248, 'E_out': 0.16196096766897528}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.16859309077381904, 'P_out': 0.6577227253698004, 'E_out': 0.1736841838564185}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.4468454540870484, 'P_out': 0.4179355745721925, 'E_out': 0.13521897134078664}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.5777421951491593, 'P_out': 0.2602968371819248, 'E_out': 0.16196096766897528}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out'

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.5440575791038958, 'P_out': 0.9660930999597694, 'E_out': 0.577964479144121}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.6755428454384402, 'P_out': 1.4399010728075883, 'E_out': 0.23564177263084984}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.6204425523299333, 'P_out': 0.338519098312379, 'E_out': 0.0410383493576867}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.3580844117022527, 'P_out': 0.6288085115265359, 'E_out': 0.013107076771209137}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.3358250816036591, 'P_out': 0.36695388236759263, 'E_out': 0.2972210360287553}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.2

Number of subsets with adequate scores: 434
Number of unblocked models: 434


In [79]:
random.seed(23)
np.random.seed(23)

In [80]:
num_adequate_subsets500_23, num_unblocked_500_23 = evaluate_reaction_subsets(model, synth_data500, test_random_reacs, 2)
print(f"Number of subsets with adequate scores: {num_adequate_subsets500_23}")
print(f"Number of unblocked models: {num_unblocked_500_23}")

[<Reaction A_in at 0x28c5dd8d0>, <Reaction B_in_out at 0x28c743990>, <Reaction P_out at 0x28c5de4d0>, <Reaction E_out at 0x28c5dc510>, <Reaction r6 at 0x28c5dd090>, <Reaction r8 at 0x28c5dd110>, <Reaction r9 at 0x28c5de750>, <Reaction r10 at 0x28c5de290>]
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.6807823782259476, 'P_out': 1.4156256203549475, 'E_out': 0.26515675787099613}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 1.3412614559953027, 'P_out': 2.307983251297816, 'E_out': 0.033278204697486914}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.717088726024776, 'P_out': 0.24484623986325996, 'E_out': 0.03806503411196312}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.43543560278559407, 'P_out': 0.5152697163358432, 'E_out': 0.049294680878565346}. Message: Inequality constraints incom

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 5.295416686592873, 'P_out': 5.809489418630538, 'E_out': 0.4859272679657926}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.5016730248790273, 'P_out': 0.44675406633306125, 'E_out': 0.051572908788229135}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.13276044831497613, 'P_out': 0.785967568141343, 'E_out': 0.0812719835441082}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.49271806343081404, 'P_out': 0.4987522186103362, 'E_out': 0.008529717959380671}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.696709149223925, 'P_out': 1.3221618476605415, 'E_out': 0.37454730156383875}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.6971654794891915, 'P_out': 1.1258170610070772, 'E_out': 0.5713484184820925}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.5727888815279653, 'P_out': 1.3930449353864733, 'E_out': 0.17974394614146155}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.015805565171162647, 'P_out': 0.632095585996556, 'E_out': 0.38370997917459637}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.4682359421097299, 'P_out': 0.3480949191787642, 'E_out': 0.18366913871148263}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.25736890671015333, 'P_out': 0.6635908723514043, 'E_out': 0.5937780343586958}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.056615474434332284, 'P_out': 1.0121421727008362, 'E_out': 0.04447330173366419}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.03041144297567415, 'P_out': 0.8541318093564592, 'E_out': 0.1762796336198114}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.5646235603844741, 'P_out': 0.252569984765799, 'E_out': 0.18280645485003927}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.5006467893705028, 'P_out': 1.2419081813346067, 'E_out': 0.25873860803625465}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.01498328958276347, 'P_out': 0.6568454140082699, 'E_out': 0.32817129640947745}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out':

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.34394574249510157, 'P_out': 0.4506364224362843, 'E_out': 0.20541783506874906}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.303767533504917, 'P_out': 0.5449455318611195, 'E_out': 0.1512869346341122}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.42795598655776035, 'P_out': 0.31827913814811337, 'E_out': 0.25376487529420416}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.5573971648495614, 'P_out': 1.4967639317792885, 'E_out': 0.0606332330704646}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.2573723693106207, 'P_out': 1.177100823696982, 'E_out': 0.08027154561379807}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0

...

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.044106063402751035, 'P_out': 1.0410597478881662, 'E_out': 0.003046315515065654}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.12826703214785296, 'P_out': 0.5378137723784351, 'E_out': 0.33391919547405546}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.3156454764030305, 'P_out': 1.229005493723153, 'E_out': 0.08663998268015123}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.14219623932534625, 'P_out': 0.9113807381508978, 'E_out': 0.2308155011750727}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 1.0331827755091747, 'P_out': 1.1367884054605948, 'E_out': 0.8963943700489665}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 1.3412614559953027, 'P_out': 2.307983251297816, 'E_out': 0.033278204697486914}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.717088726024776, 'P_out': 0.24484623986325996, 'E_out': 0.03806503411196312}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.43543560278559407, 'P_out': 0.5152697163358432, 'E_out': 0.049294680878565346}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.5115614493640922, 'P_out': 0.46584782814697273, 'E_out': 0.0225907224889376}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.4531164595859436, 'P_out': 0.9404869113410407, 'E_out': 0.5126295482449101}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out':

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.28605271429821316, 'P_out': 0.5053704330069594, 'E_out': 0.2085768526944688}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.35909695555625687, 'P_out': 0.5266605818286052, 'E_out': 0.11424246261463566}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.2792089621833846, 'P_out': 0.8568675486431875, 'E_out': 0.4223414135393443}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.3549418944121736, 'P_out': 0.8159856902821712, 'E_out': 0.5389562041296458}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.6985993420398365, 'P_out': 0.9185586724433551, 'E_out': 0.7800406695959174}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.0

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.0031030624837028583, 'P_out': 0.7260134500958907, 'E_out': 0.2708834874204841}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.011213337253032318, 'P_out': 0.67736155309309, 'E_out': 0.33385178416002986}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.10309224071569376, 'P_out': 1.0533704158245492, 'E_out': 0.04972182489126631}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.796010840819656, 'P_out': 1.4624638103212664, 'E_out': 0.33354703049854795}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.29903807156959944, 'P_out': 1.1647803805668195, 'E_out': 0.1342576910028877}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 

Number of subsets with adequate scores: 439
Number of unblocked models: 439


## Evaluation model reconstruction taking into account internal flux values

In [81]:
import cobra
from cobra.sampling import sample

import pandas as pd


# Function to normalize a row based on a normalization id, excluding boundary reactions
def normalize_row_internal(row, normalization_id):
    normalization_factor = row[normalization_id]
    # Exclude boundary reactions (columns starting with "EX_" or ending with "_out")
    filtered_row = {col: value for col, value in row.items() if not (col.startswith("EX_"))}
    return {col: (value / normalization_factor) if normalization_factor != 0 else 0.0
            for col, value in filtered_row.items()}


def create_synthetic_datapoints_internal(model, amount, normalization_id):
    s = sample(model, amount)
    
    normalized_data = [normalize_row_internal(row, normalization_id) for _, row in s.iterrows()]
    
    return normalized_data

In [82]:
random.seed(42)
np.random.seed(42)

In [83]:
synth_data_internal42 = create_synthetic_datapoints_internal(model_data_gen, 20, "A_in")

print(synth_data_internal42)

[{'A_in': 1.0, 'B_in_out': -0.6540411831851014, 'P_out': 0.3088898932341168, 'E_out': 0.03706892358077929, 'r5': 0.48304711872185313, 'r6': 0.47988395769736625, 'r7': 0.03706892358077929, 'r8': -0.4428150341165866, 'r9': 0.2718209696533385, 'r10': 0.03706892358077929}, {'A_in': 1.0, 'B_in_out': 0.30874597396374015, 'P_out': 1.2987749953646983, 'E_out': 0.00997097859904201, 'r5': 0.48780920279992035, 'r6': 0.5022198186010375, 'r7': 0.00997097859904201, 'r8': -0.4922488400019966, 'r9': 1.2888040167656563, 'r10': 0.00997097859904201}, {'A_in': 1.0, 'B_in_out': 0.005400243812449617, 'P_out': 0.8132712619954356, 'E_out': 0.1921289818170098, 'r5': 0.6811500771008573, 'r6': 0.1267209410821313, 'r7': 0.1921289818170098, 'r8': 0.06540804073487932, 'r9': 0.621142280178427, 'r10': 0.1921289818170098}, {'A_in': 1.0, 'B_in_out': 0.2652811938103047, 'P_out': 1.1311666023291493, 'E_out': 0.13411459148115204, 'r5': 0.07413940191309873, 'r6': 0.7917460066057482, 'r7': 0.13411459148115204, 'r8': -0.6576

In [84]:
random.seed(17)
np.random.seed(17)

In [85]:
num_adequate_subsets_internal17 = evaluate_reaction_subsets(model, synth_data_internal42, test_random_reacs, 2)
print(f"Number of subsets with adequate scores: {num_adequate_subsets_internal17}")

[<Reaction A_in at 0x172f96410>, <Reaction B_in_out at 0x172f978d0>, <Reaction P_out at 0x172f95550>, <Reaction E_out at 0x172f945d0>, <Reaction r5 at 0x172f94150>, <Reaction r6 at 0x172f976d0>, <Reaction r7 at 0x172f95e90>, <Reaction r10 at 0x172f97110>]
Number of subsets with adequate scores: (14, 14)


In [86]:
random.seed(23)
np.random.seed(23)

In [87]:
num_adequate_subsets_internal23 = evaluate_reaction_subsets(model, synth_data_internal42, test_random_reacs, 2)
print(f"Number of subsets with adequate scores: {num_adequate_subsets_internal23}")

[<Reaction A_in at 0x28c840390>, <Reaction B_in_out at 0x28c843b90>, <Reaction P_out at 0x28c840a50>, <Reaction E_out at 0x28c841750>, <Reaction r6 at 0x28d91f4d0>, <Reaction r8 at 0x28f2d1ed0>, <Reaction r9 at 0x28ccd6f50>, <Reaction r10 at 0x28d91de90>]
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.6540411831851014, 'P_out': 0.3088898932341168, 'E_out': 0.03706892358077929, 'r6': 0.47988395769736625, 'r8': -0.4428150341165866, 'r9': 0.2718209696533385, 'r10': 0.03706892358077929}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.30874597396374015, 'P_out': 1.2987749953646983, 'E_out': 0.00997097859904201, 'r6': 0.5022198186010375, 'r8': -0.4922488400019966, 'r9': 1.2888040167656563, 'r10': 0.00997097859904201}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.005400243812449617, 'P_out': 0.8132712619954356, 'E_out': 0.1921289818170098, 'r6': 0.126

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.6540411831851014, 'P_out': 0.3088898932341168, 'E_out': 0.03706892358077929, 'r6': 0.47988395769736625, 'r8': -0.4428150341165866, 'r9': 0.2718209696533385, 'r10': 0.03706892358077929}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.30874597396374015, 'P_out': 1.2987749953646983, 'E_out': 0.00997097859904201, 'r6': 0.5022198186010375, 'r8': -0.4922488400019966, 'r9': 1.2888040167656563, 'r10': 0.00997097859904201}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.005400243812449617, 'P_out': 0.8132712619954356, 'E_out': 0.1921289818170098, 'r6': 0.1267209410821313, 'r8': 0.06540804073487932, 'r9': 0.621142280178427, 'r10': 0.1921289818170098}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.2652811938103047, 'P_out': 1.1311666023291493, 'E_ou

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.6540411831851014, 'P_out': 0.3088898932341168, 'E_out': 0.03706892358077929, 'r6': 0.47988395769736625, 'r8': -0.4428150341165866, 'r9': 0.2718209696533385, 'r10': 0.03706892358077929}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.30874597396374015, 'P_out': 1.2987749953646983, 'E_out': 0.00997097859904201, 'r6': 0.5022198186010375, 'r8': -0.4922488400019966, 'r9': 1.2888040167656563, 'r10': 0.00997097859904201}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.005400243812449617, 'P_out': 0.8132712619954356, 'E_out': 0.1921289818170098, 'r6': 0.1267209410821313, 'r8': 0.06540804073487932, 'r9': 0.621142280178427, 'r10': 0.1921289818170098}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.2652811938103047, 'P_out': 1.1311666023291493, 'E_ou

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.30874597396374015, 'P_out': 1.2987749953646983, 'E_out': 0.00997097859904201, 'r6': 0.5022198186010375, 'r8': -0.4922488400019966, 'r9': 1.2888040167656563, 'r10': 0.00997097859904201}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.0998574516127496, 'P_out': 0.7709618332260167, 'E_out': 0.12918071516123206, 'r6': 0.2228545543427832, 'r8': -0.09367383918155095, 'r9': 0.6417811180647839, 'r10': 0.12918071516123206}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.08908988842918208, 'P_out': 0.6742132975022708, 'E_out': 0.23669681406854368, 'r6': 0.1402190131519334, 'r8': 0.09647780091661128, 'r9': 0.43751648343372773, 'r10': 0.23669681406854368}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.0400891174833084, 'P_out': 0.6815465382102195, 

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.6540411831851014, 'P_out': 0.3088898932341168, 'E_out': 0.03706892358077929, 'r6': 0.47988395769736625, 'r8': -0.4428150341165866, 'r9': 0.2718209696533385, 'r10': 0.03706892358077929}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.30874597396374015, 'P_out': 1.2987749953646983, 'E_out': 0.00997097859904201, 'r6': 0.5022198186010375, 'r8': -0.4922488400019966, 'r9': 1.2888040167656563, 'r10': 0.00997097859904201}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.005400243812449617, 'P_out': 0.8132712619954356, 'E_out': 0.1921289818170098, 'r6': 0.1267209410821313, 'r8': 0.06540804073487932, 'r9': 0.621142280178427, 'r10': 0.1921289818170098}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.5837763632344605, 'P_out': 0.21694758758817229, 'E_

Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.17459207627886428, 'P_out': 0.4904866083975133, 'E_out': 0.3349213153236113, 'r6': 0.10971632491689771, 'r8': 0.22520499040671785, 'r9': 0.15556529307389994, 'r10': 0.3349213153236113}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.17459207627886428, 'P_out': 0.4904866083975133, 'E_out': 0.3349213153236113, 'r6': 0.10971632491689771, 'r8': 0.22520499040671785, 'r9': 0.15556529307389994, 'r10': 0.3349213153236113}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': -0.6540411831851014, 'P_out': 0.3088898932341168, 'E_out': 0.03706892358077929, 'r6': 0.47988395769736625, 'r8': -0.4428150341165866, 'r9': 0.2718209696533385, 'r10': 0.03706892358077929}. Message: Inequality constraints incompatible
Optimization failed for data point: {'A_in': 1.0, 'B_in_out': 0.30874597396374015, 'P_out': 1.2987749953646983, 

Number of subsets with adequate scores: (18, 18)
